In [54]:
from bitcoinrpc.authproxy import AuthServiceProxy
import pandas as pd

#### In BTC Node, I have already imported these addreeses under 'test' label to watch only addresses and rescanned the wallet for all the transactions.


In [57]:
rpc_connection = AuthServiceProxy("http://%s:%s@ec2-3-81-20-12.compute-1.amazonaws.com:8332"%("test", "test"))

In [58]:
labelled_addr = rpc_connection.getaddressesbylabel("test")
len(labelled_addr)


899

In [59]:
df = pd.DataFrame()
count_val = 1000
skip_val = 0
while(True):
    get_txs = rpc_connection.listtransactions("*", count_val, skip_val, True) # see API docs for arg details
    if get_txs == []:
        break
    df = df.append(pd.DataFrame(get_txs), ignore_index=True)
    print(f"fetched {len(df)} txs")
    skip_val+=count_val

df

fetched 717 txs


,involvesWatchonly,address,category,amount,label,vout,confirmations,blockhash,blockheight,blockindex,blocktime,txid,walletconflicts,time,timereceived,bip125-replaceable,fee,abandoned
0,True,3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja,receive,50.00000000,test,0,64788,0000000000000000000a1c5c75de52f3fda0b444b2d956...,617518,872,1581789478,0206a1c8723c7a0ef894354da3a4cdfa9787541c759a18...,[],1581789478,1619660717,no,NaN,NaN
1,True,3JNZyt1ocETf9oHscQfx17XNqY3odHAjaR,receive,50.00000000,test,1,64788,0000000000000000000a1c5c75de52f3fda0b444b2d956...,617518,1082,1581789478,784b529b29b054e15637ec0ceb7dcc56e34f0737ccb9f7...,[],1581789478,1619660717,no,NaN,NaN
2,True,3QhHo5Y2odxP6ysLya5YvUQZMxyFDxyYjL,send,-99.99992736,NaN,0,64777,00000000000000000005613da8ffa2191bf7b851446079...,617529,36,1581792148,f9aaddbc3a7fd1079e4418dc2b0a8be76caa42c4127819...,[],1581792148,1619660717,no,-0.00007264,False
3,True,3JjtnDXknvY7HWidn5wt5BdZsRdXCGEt8y,receive,10.00000000,test,0,63167,000000000000000000116e16b59bcc11133e158d28ad95...,619139,521,1582764816,96783e3b6e405410a4c1677dc98b09836abafccbfe33e1...,[],1582764816,1619660755,no,NaN,NaN
4,True,32RLGxLjwEvcaT7r3afgZQskYfZ9bqxHHm,send,-9.99997840,NaN,0,62119,0000000000000000000d13e44844b56e399746719d415e...,620187,2961,1583348600,54f55be92d7424880981bb25c01992b6bbff13da1e47a0...,[],1583348600,1619660780,no,-0.00002160,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,True,bc1qa92ggx8gmaz9zg62wqz2kqdhl5qqxpv6ggtmvj,receive,0.00091862,test,0,509,0000000000000000000160a86ec0d39b793ef8395b0ba5...,681797,543,1620086827,bfd563ccb04b451695f218490b7de7987bc86634bf49e6...,[],1620086363,1620086363,no,NaN,NaN
713,True,bc1q5et0duz9jkl56lwsl6gq5nxuu5dskunwzexd6r,receive,1.00000000,test,0,352,0000000000000000000b3b83e51a779ec9adcfcf3a7f82...,681954,311,1620163929,fcc7359648d8ffb5498e2f0ea345d13d1d74c635a2f248...,[],1620163843,1620163843,no,NaN,NaN
714,True,bc1q5et0duz9jkl56lwsl6gq5nxuu5dskunwzexd6r,receive,0.06207628,test,1,346,00000000000000000004bfd7ebbb74b876aa8e488dc07a...,681960,313,1620165254,207a0f8dbd59f8337633b7f7a42a5837e713226c8cb40e...,[],1620165142,1620165142,no,NaN,NaN
715,True,bc1q5et0duz9jkl56lwsl6gq5nxuu5dskunwzexd6r,receive,0.17756380,test,0,344,000000000000000000065b0ebdb599864c1f7ecba417c4...,681962,768,1620166483,85f5180782b0d42cacf4ba0e7e31bc1aeba52477f93479...,[],1620165447,1620165447,no,NaN,NaN


In [60]:
hashMap = {}
for l in labelled_addr:
    hashMap[l] = set()

tx_df = pd.DataFrame()
i = 0
for index, row in df.iterrows():
    if row.category == 'receive':
        hashMap[row.address].add(row.txid)
    
    # for send category, check desired addr in vin of tx
    else:
        raw_data = rpc_connection.getrawtransaction(row.txid, True)
        vins = raw_data['vin']
        processed = []
        for vin in vins:
            if vin['txid'] not in processed:
                real_vin = rpc_connection.getrawtransaction(vin['txid'], True)['vout'][vin['vout']]
                for addr in real_vin['scriptPubKey']['addresses']:
                    if addr in labelled_addr and row.txid not in hashMap[addr]:
                        hashMap[addr].add(row.txid)
                        print(f"found {addr} in {row.txid}")
                processed.append(vin['txid'])
    i = i+1
    print(f"{i}/{len(df)}")
hashMap

1/717
2/717
found 3DKcHmHdqcUDAX2Yi2sjX9Hauam25Nk5Ja in f9aaddbc3a7fd1079e4418dc2b0a8be76caa42c4127819c75d3c7db483bb7692
found 3JNZyt1ocETf9oHscQfx17XNqY3odHAjaR in f9aaddbc3a7fd1079e4418dc2b0a8be76caa42c4127819c75d3c7db483bb7692
3/717
4/717
found 3JjtnDXknvY7HWidn5wt5BdZsRdXCGEt8y in 54f55be92d7424880981bb25c01992b6bbff13da1e47a08699604116d2ca90de
5/717
6/717
7/717
8/717
found 336Cqh6hp7qXtJtU26BAPFWW1v2bBymrej in 6e34edc5a5b63aedd3547bf08448cd2701ba6b1ab923547ed3e7fc5b94f832be
found 3J9rQSzvNThPdN5FDbJAD3N8Wdce1KF6Ms in 6e34edc5a5b63aedd3547bf08448cd2701ba6b1ab923547ed3e7fc5b94f832be
9/717
10/717
11/717
12/717
found 336Cqh6hp7qXtJtU26BAPFWW1v2bBymrej in 59c53fd1c4bbeb36d77459b2b38b8257b1093efe7a76e0797bf234de90c78d19
found 3JPVMgc4NxK5UTRnPiihtUyFzWwsVqjzJQ in 59c53fd1c4bbeb36d77459b2b38b8257b1093efe7a76e0797bf234de90c78d19
13/717
14/717
15/717
16/717
found 3EWxsS8SBYKmYn3aPQ2uu84iehK8J5MGNW in 53dc175ac0eef011e090f428ff65697ed2bfcdcfd70e0041f7d2c85435fa4e5c
17/717
18/717
19/717
20/7

found 14NhPc2N83mxuYDjECrYFoTNgHhTkxdnga in 1299693121e3afad0c107c43ec95f9fe783e2205d427c5b199e301e59b3f0006
found 1NjKYd95vjVZFb7rZWuyZ8ha2Kq9cxe6Jg in 1299693121e3afad0c107c43ec95f9fe783e2205d427c5b199e301e59b3f0006
found 13qF2huSpD4Vim62ChBFbUjtxWQ1zSCE4W in 1299693121e3afad0c107c43ec95f9fe783e2205d427c5b199e301e59b3f0006
found 1HN982uCcmHBAAx6w7hn3g9N5PwzMa9tKT in 1299693121e3afad0c107c43ec95f9fe783e2205d427c5b199e301e59b3f0006
144/717
145/717
146/717
147/717
148/717
149/717
150/717
151/717
152/717
153/717
154/717
155/717
156/717
157/717
158/717
found 1PRWdkAMyfU6ntKjBppXGGHqVh352iai4x in 158f0946e95c72d87d25294f896e5cc9f46dd0d6256a73c45227642454a38599
found 12tDcSy4qHiMsK2VGbh6ygoC3W1dtuu9w6 in 158f0946e95c72d87d25294f896e5cc9f46dd0d6256a73c45227642454a38599
found 1KvBmhCk7XzXmdD8PGF5zPYrEX37iPWiLv in 158f0946e95c72d87d25294f896e5cc9f46dd0d6256a73c45227642454a38599
found 1WXtgi4sX3PnQb9XXq8qnParoGpxbbRhJ in 158f0946e95c72d87d25294f896e5cc9f46dd0d6256a73c45227642454a38599
found 18T

found bc1qwxewtqydlteav53hdqp8kcdpexe5edxlc69ke0 in 1a116958aa07ca1fc1366d97efa27743ef44e94bd01ae93de162816260615c18
found bc1qa92ggx8gmaz9zg62wqz2kqdhl5qqxpv6ggtmvj in 1a116958aa07ca1fc1366d97efa27743ef44e94bd01ae93de162816260615c18
found bc1qvmhp2u003t30860x5gwrv5yu4jjgynng5tc6vv in 1a116958aa07ca1fc1366d97efa27743ef44e94bd01ae93de162816260615c18
found bc1q8t4r0rdmggt6upjx2lfaue8p6kmnxugcwymlhd in 1a116958aa07ca1fc1366d97efa27743ef44e94bd01ae93de162816260615c18
found 1Ktzy8apiv5gwMmMVXcvJ8Yh4nbrf9jGmp in 1a116958aa07ca1fc1366d97efa27743ef44e94bd01ae93de162816260615c18
found 1GA8ynymBSUrxMo1ZZsazVvwio9jjjz8Pz in 1a116958aa07ca1fc1366d97efa27743ef44e94bd01ae93de162816260615c18
found 1HN982uCcmHBAAx6w7hn3g9N5PwzMa9tKT in 1a116958aa07ca1fc1366d97efa27743ef44e94bd01ae93de162816260615c18
324/717
325/717
found bc1ques95tupg9xa3h6hc8tsj40kdmdl9y8tx7fy0g in a5a6f250e94bf3946b7556dcfb3e4b817a458ac66e7e0d73b9372814e4a8c331
326/717
327/717
328/717
329/717
330/717
found bc1qwpnlc4tvlprqs20jfmft6e

found bc1qqfdcvgu5w6nvanpgs49aznvh6d62tsrvjgedjv in 2504347134a4653694171631135d621638f2bffa3425d353702d126126a016cf
found bc1qnllmvdqkzmzmvs0cqaytldyz9sd4pmsxmzq54m in 2504347134a4653694171631135d621638f2bffa3425d353702d126126a016cf
439/717
440/717
441/717
442/717
443/717
444/717
445/717
found bc1qdh0yfe4vektndqsk630a2nc30n0xwre9lu54zl in eeffcf30b437de3a8c4d3b6928778fbed764aefa722454c2d8a2c476eb123e8a
446/717
447/717
found bc1qdh0yfe4vektndqsk630a2nc30n0xwre9lu54zl in ece39e40a56cda667202e47812dc4cccc8ca515301fa28cf57b998a29bb23c6e
448/717
449/717
450/717
451/717
found bc1qpdwqw8d499lf830rmfyf9qzlsmagrz7xm6gf4j in 057c8f5274e9e361a87f7c642d23be7945051379d7719caf5f39262bf6ca18a2
found bc1qa92ggx8gmaz9zg62wqz2kqdhl5qqxpv6ggtmvj in 057c8f5274e9e361a87f7c642d23be7945051379d7719caf5f39262bf6ca18a2
452/717
453/717
found 34sPujxTo5GTnF8nySK8qakCkNm5HQgS7M in 184abc1d33f1c24a7ec79891f8500f7a69bc6c4bdd70e19eb6d2ca07bdac562f
454/717
455/717
456/717
found 34sPujxTo5GTnF8nySK8qakCkNm5HQgS7M in c

found bc1qa92ggx8gmaz9zg62wqz2kqdhl5qqxpv6ggtmvj in 38dd5847974994c3d4066634bb8f2c0b066919a2d85ac3c95e422b106c7ad81f
found bc1qfdfcej8vvsau0mq087xugdj0l0c5085g8wuwe8 in 38dd5847974994c3d4066634bb8f2c0b066919a2d85ac3c95e422b106c7ad81f
found bc1qaeltrssj6eyx8j4rkup67j68y88psvqlpwzhjz in 38dd5847974994c3d4066634bb8f2c0b066919a2d85ac3c95e422b106c7ad81f
found bc1q7yqnp965hustpzprw4sxtwudl2cmfuzpjxqqkr in 38dd5847974994c3d4066634bb8f2c0b066919a2d85ac3c95e422b106c7ad81f
found bc1qk9jrfyd5gpu5nqfr5u9w7kdtmgapl7v7j07ct2 in 38dd5847974994c3d4066634bb8f2c0b066919a2d85ac3c95e422b106c7ad81f
found bc1qfp327emcven6qq8krufg3segph0jxwspdkdx68 in 38dd5847974994c3d4066634bb8f2c0b066919a2d85ac3c95e422b106c7ad81f
604/717
605/717
606/717
607/717
found bc1qfqpdeep2yr60sewzeud2faa6958jppraar82xs in 336ae43ef172ada3e6d69856babbf6d86453cc514020c8642ca430743f901b39
608/717
609/717
610/717
found 3L9M1wPpy5vvntsTvt48AgJQm1ubaztmrT in 8b132e70267f7d401e787889741d59d4de6720fa4afa95de99da6ffa4af34485
611/717
612/717


{'11t5PBBjzbWYuD3BJs9B7rGYtPfUqvGp3': set(),
 '113LaspLBbvVMidszwDedJ7eAJzzvYj1xy': set(),
 '17uwRvQvoRp5RujVUXDu1eYQBuXD11vsV': set(),
 '18qXCZ1vSJoiyxmSBfWmH2qPDjgdamf67': set(),
 '1BH6x6qonJF42avwvDVJ9AWevEYCUA4Ay': set(),
 '1DTzaiWLFThgZsoCzzfkKgiinRtXqj4ho': set(),
 '1FfRHinBanP8L4FaCMoJdte1XjHpvBBLX': set(),
 '1HwNmm7U5G8xxosguubusW6Ma35YV19S5': set(),
 '1KMomEQrYBgiuZRvPKAu9PBHLP17y5Xbw': set(),
 '1M4mZKFtjodhP3ktdLiYj76juoZVnug4P': set(),
 '1W8qUpgovGTpVuma9PhGJcEnCzhZizsjq': set(),
 '1WXtgi4sX3PnQb9XXq8qnParoGpxbbRhJ': {'0a1d4bbd5de2a3f4fda8462e7a21b9af50146e1c80993abfbf8308334658bac3',
  '0f02cf420e49af891f50735ce5bb1cc0a2437ff03170a58c6f26e544112078fc',
  '158f0946e95c72d87d25294f896e5cc9f46dd0d6256a73c45227642454a38599',
  '4e4cab1af063ecf6825c2e12aba543c28ce06381b48ecb66e9b8200be9295656',
  '65e45a4414eb4ed578de8bea7b880b4ad8efc59948fde407c6143490e93d9036',
  '6d35c1c3b88c4469814e5b2a4c2ba4f00da711706cae142e27b1f68d7b188fe8',
  '8e6e20f68e476343e59af063edf4a06c09ee4934b3f8